In [1]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torch.nn.functional as F

import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import sys
sys.path.append('/kaggle/input/senfiles')

In [3]:
from models import ModernTransUNetV2
from misc import initialize_config, rle_encode
from data import initialize_data, TestDataset, test_collate
from loss import CombinedLoss
from wheels import train, evaluate

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [4]:
# Given a directory containing the data folders
ROOT ='/kaggle/input/'

config = initialize_config(ROOT)
config["weights"] = '/kaggle/input/weights/mk2p2.pth'

In [5]:
config["model_params"]["resnet_settings"]["blocks"] = [3, 4, 9]
#config["train_params"]["kidney_names"] = ['kidney_3_dense', 'kidney_3_sparse', 'kidney_1_voi']
#config["validation_params"]["kidney_names"] = ['kidney_1_dense', 'kidney_1_voi', 'kidney_2', 'kidney_3_sparse']
config["num_workers"] = 4
config["batch_size"] = 4
config["epochs"] = 90
#config["threshold"] = .1

config["model_params"]["parallel_settings"]["trunk_blocks"] = 3

config["model_params"]["resnet_settings"]["normalization"] = "group"
config["model_params"]["leakyReLU"] = True
config["model_params"]["instanceNorm"] = False
config["model_params"]["trunk/res_channels"] = 32
config["model_params"]["parallel_settings"]["flag"] = True
config["model_params"]["parallel_settings"]["concatenate"] = True
config["model_params"]["parallel_settings"]["branch_blocks"] = 1


config["model_params"]["transformer_params"]["num_layers"] = 12
config["model_params"]["transformer_params"]["num_heads"] = 12
config["model_params"]["transformer_params"]["hidden_dim"] = 768
config["model_params"]["transformer_params"]["mlp_dim"] = 3072

config["model_params"]["interpolation_settings"]["flag"] = True
config["model_params"]["interpolation_settings"]["topper"] = True
config["model_params"]["interpolation_settings"]["root_out"] = 16
config["model_params"]["interpolation_settings"]["blocks"] = 1
config["model_params"]["interpolation_settings"]["normalization"] = 'group'

In [6]:
config["model_params"]["resnet_settings"]["blocks"] = [3, 4, 9]

#config["train_params"]["kidney_names"] = ['kidney_2', 'kidney_1_voi']
#config["validation_params"]["kidney_names"] = ['kidney_1_dense', 'kidney_3_dense', 'kidney_3_sparse', 'kidney_1_voi']
config["train_params"]["kidney_names"] = ['kidney_1_voi', 'kidney_3_dense', 'kidney_3_sparse']
config["validation_params"]["kidney_names"] = ['kidney_1_dense', 'kidney_2', 'kidney_1_voi']

config["num_workers"] = 4
config["batch_size"] = 4
config["epochs"] = 90
#config["threshold"] = .1

config["model_params"]["parallel_settings"]["trunk_blocks"] = 1

config["model_params"]["resnet_settings"]["normalization"] = "group"
config["model_params"]["leakyReLU"] = True
config["model_params"]["instanceNorm"] = False
config["model_params"]["trunk/res_channels"] = 32
config["model_params"]["parallel_settings"]["flag"] = True
config["model_params"]["parallel_settings"]["concatenate"] = True
config["model_params"]["parallel_settings"]["branch_blocks"] = 1


config["model_params"]["transformer_params"]["num_layers"] = 12
config["model_params"]["transformer_params"]["num_heads"] = 16
config["model_params"]["transformer_params"]["hidden_dim"] = 768
config["model_params"]["transformer_params"]["mlp_dim"] = 3072

config["model_params"]["interpolation_settings"]["flag"] = True
config["model_params"]["interpolation_settings"]["topper"] = True
config["model_params"]["interpolation_settings"]["root_out"] = 16
config["model_params"]["interpolation_settings"]["blocks"] = 1
config["model_params"]["interpolation_settings"]["normalization"] = 'group'

In [7]:
model = ModernTransUNetV2(config["model_params"], config["device"]).to(config["device"])

In [8]:
if config["weights"]:
    components = torch.load(config["weights"])
    model.load_state_dict(components["model_state_dict"])



In [9]:
'''# Check to see if the weights were loaded correctly
train_dl, validity_dl = initialize_data(config)
avg_validation_loss, individual_losses, precision, recall, f1 = evaluate(model, validity_dl, None, config['device'])

print('Validation Loss: {:.4f} | Dice Loss: {:.4f} | BCE Loss: {:.4f}'
              .format(avg_validation_loss, individual_losses[0], individual_losses[1]))
print("Precision: {:.4f} | Recall: {:.4f} | F1: {:.4f}".format(precision, recall, f1))'''

'# Check to see if the weights were loaded correctly\ntrain_dl, validity_dl = initialize_data(config)\navg_validation_loss, individual_losses, precision, recall, f1 = evaluate(model, validity_dl, None, config[\'device\'])\n\nprint(\'Validation Loss: {:.4f} | Dice Loss: {:.4f} | BCE Loss: {:.4f}\'\n              .format(avg_validation_loss, individual_losses[0], individual_losses[1]))\nprint("Precision: {:.4f} | Recall: {:.4f} | F1: {:.4f}".format(precision, recall, f1))'

In [10]:
test_config = config["test_params"]

# Iterate through the directories in the test folder, yielding the path names
test_data = TestDataset(test_config["test_data_path"], test_config["test_transforms"])
submission = test_data.getSubmission()
test_loader = DataLoader(test_data, batch_size=4, shuffle=True, collate_fn=test_collate)

model.eval()

with torch.no_grad():
    for i, tdata in enumerate(test_loader):
        test_ids, small, medium, large, original = tdata
        test_outputs = model(small.to(config["device"]), medium.to(config["device"]), large.to(config["device"]), original)

        for j in range(len(test_outputs)):
            mask = torch.where(test_outputs[j] > config["threshold"], 1, 0)
            encoded_test_mask = rle_encode(np.array(mask.cpu()).squeeze())

            location = submission.loc[submission['id'] == test_ids[j]].index[0]
            submission.at[location, 'rle'] = encoded_test_mask

submission.to_csv('./submission.csv', index=False)